Robotic Systems 1 - Homework 2

In [ ]:
import numpy as np # Linear Algebra
import pinocchio as pin # Pinocchio library
import os

from pinocchio.robot_wrapper import RobotWrapper
from pinocchio.visualize import MeshcatVisualizer

VISUALIZER = MeshcatVisualizer

In [ ]:
def load_franka():
  current_path = os.path.abspath('') # where the folder `robot` is located at
  robot_path = os.path.join(current_path, "robot")

  robot = RobotWrapper.BuildFromURDF(os.path.join(robot_path, "franka.urdf"), package_dirs = robot_path)

  return robot

In [ ]:
# function to compute FK for all frames/joints
def fk_all(model, data, q, v = None):
    if v is not None:
        pin.forwardKinematics(model, data, q, v) # FK and Forward Velocities
    else:
        pin.forwardKinematics(model, data, q) # FK
    pin.updateFramePlacements(model, data) # Update frames

In [ ]:
def step_world(model, data, t_q, t_qdot, t_tau, t_dt):
  
  qdotdot = pin.aba(model, data, t_q, t_qdot, t_tau)
  qdot = t_qdot+ qdotdot*t_dt
  q = pin.integrate(model,t_q, qdot*t_dt)
  
  return q, qdot

In [ ]:
robot = load_franka()

model = robot.model
data = robot.data

robot.setVisualizer(VISUALIZER())
robot.initViewer()
robot.loadViewerModel("pinocchio")

robot.display(pin.randomConfiguration(model))


# Simulation

In [ ]:
T = 10.
dt = 0.01
K = int(T/dt) + 1

q = pin.randomConfiguration(model)
qdot = np.zeros(model.nv)
tau = np.zeros(model.nv)
end_effector = model.getFrameId("panda_ee")

for k in range(K):

  q ,_  = step_world(model, data, q, qdot, tau, dt)
  robot.display(q)

  T_wd = data.oMf[end_effector]


In [ ]:
print(model.lowerPositionLimit)
print(model.upperPositionLimit)
print(model.velocityLimit)

## Task-Space Trajectories

I should implement 4 different profiles. I decided to do them in 4 different ways. So we have:
- Screw Path
- Decoupled trajectory
- The above with third order polynomials for time scaling


Questions:
- orientation and translation error should be computed separately in the controller -> straight lines in the task space (not on the SO3 manifold) but what if the trajectory is computed on the manifold? (screw path(on the SO3 manifold) and decoupled rot and trans)

In [ ]:
# A = pin.SE3.Identity()
A = pin.SE3.Identity()
B = pin.SE3.Random()

M = pin.SE3.Interpolate(A,B,0.5)
print(M)

# this is the same as pin.SE3.Interpolate()
def interpolate_rotation(A, B, s): 
  return pin.exp(pin.log(A.inverse()*B)*s)

# translation Position decoupling
def interpolate_translation(start,goal,s):
  return start + s*(goal-start)

p = interpolate_translation(A.translation,B.translation,0.5)

print(p)
M.translation = p
print(M)

In [ ]:

def get_Twd(start, goal, time, T):
  return pin.SE3.Interpolate(start, goal, max(1, time/T))

# Task-Space Controller